In [23]:
import numpy as np
import pandas as pd

In [24]:
def path(name, type_):
    return "~ksuma/isomirs/" + name + "_isomir/output-IsoMiRmap_v5-"+type_+"-isomiRs.expression.txt"

In [25]:
def get_table(name, type_, lst, dig):
    df = pd.read_fwf(path(name, type_), delimiter = '/t')
    df = df['## Table of '+type_+'-isomiRs.'].str.split('\t', expand = True)
    df.columns = df.loc[dig]
    df = df.where(df[df.columns[-1]] == '').dropna()
    df.index = 1 + np.arange(0, len(df))
    df['RPM '+name] = df['RPM**'] 
    return df[['License Plate', 'IsomiR sequence', 'RPM ' + name]]

In [26]:
def get_type(name, table, typ):
    table[name+' type'] = [typ if table['RPM '+name].loc[i] else None for i in range(table.shape[0])]
    return table

### Ambiguous

In [27]:
lst = [2, 3, 4, 6, 7, 8, 9]
ELOV5_amb = get_table("ELOV5", "ambiguous", lst, 5)
IGFBP6_amb = get_table("IGFBP6", "ambiguous", lst, 5)
LUC_amb = get_table("LUC", "ambiguous", lst, 5)
ambiguous = pd.concat([ELOV5_amb, IGFBP6_amb, LUC_amb])
ambiguous = ambiguous.groupby('License Plate').agg({'IsomiR sequence' : 'first', 'RPM ELOV5':'first',
                             'RPM IGFBP6': 'first', 'RPM LUC':'first' }).reset_index()

ambiguous.to_csv('ambiguous', sep = '\t')

ambiguous = get_type('ELOV5', ambiguous, 'ambiguous')
ambiguous = get_type('IGFBP6', ambiguous, 'ambiguous')
ambiguous = get_type('LUC', ambiguous, 'ambiguous')
# ambiguous

### Exclusive

In [17]:
ELOV5_excl = get_table("ELOV5", "exclusive", lst, 5)
IGFBP6_excl = get_table("IGFBP6", "exclusive", lst, 5)
LUC_excl = get_table("LUC", "exclusive", lst, 5)
exclusive = pd.concat([ELOV5_excl, IGFBP6_excl, LUC_excl])
exclusive = exclusive.groupby('License Plate').agg({'IsomiR sequence' : 'first', 'RPM ELOV5':'first',
                             'RPM IGFBP6': 'first', 'RPM LUC':'first' }).reset_index()
exclusive.to_csv('exclusive', sep = '\t')

exclusive = get_type('ELOV5', exclusive, 'exclusive')
exclusive = get_type('IGFBP6', exclusive, 'exclusive')
exclusive = get_type('LUC', exclusive, 'exclusive')
# exclusive

#### SNP

In [16]:
lst = [2, 3, 5, 6, 7, 8, 9]
ELOV5_snps = get_table("ELOV5", "snps", lst, 5)
IGFBP6_snps = get_table("IGFBP6", "snps", lst, 5)
LUC_snps = get_table("LUC", "snps", lst, 5)
snps = pd.concat([ELOV5_snps, IGFBP6_snps, LUC_snps])
snps = exclusive.groupby('License Plate').agg({'IsomiR sequence' : 'first', 'RPM ELOV5':'first',
                             'RPM IGFBP6': 'first', 'RPM LUC':'first' }).reset_index()
snps.to_csv('snps', sep = '\t')

#### for all types

In [12]:
set(ambiguous['IsomiR sequence']) & set(exclusive['IsomiR sequence'])

set()

In [13]:
alls = pd.concat([ambiguous, ambiguous])
alls.sample(30)
alls.to_csv('all_types', sep = '\t')